In [1]:
from pyspark.sql import SparkSession, functions as F
import findspark
import pandas as pd
import boto3
from datetime import datetime
current_ts = datetime.now().strftime('%Y%m%d')


pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)

from pyspark.sql.types import *
from pyspark.sql.functions import col, json_tuple, to_json, from_json, explode_outer

# /opt/manual/spark: this is SPARK_HOME path
findspark.init("/opt/manual/spark")

#spark = SparkSession.builder.config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.2.0").getOrCreate()
# spark & minio configuration on s3
spark = (SparkSession.builder
         .appName("Spark Minio Test")
         .config("spark.hadoop.fs.s3a.endpoint", "http://172.18.0.2:9000")
         .config("spark.hadoop.fs.s3a.access.key", 'root')
         .config("spark.hadoop.fs.s3a.secret.key", 'root12345')
         .config("spark.hadoop.fs.s3a.path.style.access", True)
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0,io.delta:delta-core_2.12:1.0.0")
         # .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0")
         .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
         .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
         .getOrCreate())

# Delta ve s3a configlerinin ayni sparksession altinda calistirilmasi icin, config.jars.packages ',' ile birlestirilmeli!!!
# `.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0,io.delta:delta-core_2.12:1.0.0")` seklinde uygundur

:: loading settings :: url = jar:file:/opt/manual/spark-3.1.1-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/train/.ivy2/cache
The jars for the packages stored in: /home/train/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c43c6bd4-9995-4b56-a5b2-28143ec7cb57;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 1063ms :: artifacts dl 22ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	com.ibm.icu#icu4j;58

In [2]:
from delta.tables import *

In [3]:
credits = (spark.read.format("parquet")
           .option("header", True)
           .option("inferSchema", True)
           .load(f"s3a://tmdb-bronze/credits/credits_part_20221023-230801.parquet"))
credits.show(10, truncate=False)

credits.printSchema()

AnalysisException: Path does not exist: s3a://tmdb-bronze/credits/credits_part_20221023-230801.parquet

In [18]:
# write delta table to silver bucket
credits.write.mode("overwrite").format("delta").save(f"s3a://tmdb-silver/{current_ts}")

In [42]:
customers = spark.createDataFrame([(10001504, "Hasan Şahintepesi", "Çankırı", "Eldivan", True, "1992-09-01", None),
                                    (10001505, "Tuncay Çadırcı", "Ankara", "Keçiören", True, "1992-10-12", None),
                                  (10001506, "Melahat Bakır", "İstanbul", "Beykoz", True, "1992-08-26", None),
                                   (10001526, "Sultan Balcı", "Muğla", "Bodrum", True, "1992-09-26", None),
                                  (10001530, "Dudu Karagölet", "Yozgat", "Sorgun", False, "1992-08-11", "1993-08-25"),
                                  (10001518, "Burcu Vataneri", "Kırşehir", "Mucur", False, "1992-08-22", "1993-06-21")],
                                   ["Id", "personName", "state", "province", "still_here", "join_date", "leave_date"])

In [43]:
customers.show()

+--------+-----------------+--------+--------+----------+----------+----------+
|      Id|       personName|   state|province|still_here| join_date|leave_date|
+--------+-----------------+--------+--------+----------+----------+----------+
|10001504|Hasan Şahintepesi| Çankırı| Eldivan|      true|1992-09-01|      null|
|10001505|   Tuncay Çadırcı|  Ankara|Keçiören|      true|1992-10-12|      null|
|10001506|    Melahat Bakır|İstanbul|  Beykoz|      true|1992-08-26|      null|
|10001526|     Sultan Balcı|   Muğla|  Bodrum|      true|1992-09-26|      null|
|10001530|   Dudu Karagölet|  Yozgat|  Sorgun|     false|1992-08-11|1993-08-25|
|10001518|   Burcu Vataneri|Kırşehir|   Mucur|     false|1992-08-22|1993-06-21|
+--------+-----------------+--------+--------+----------+----------+----------+



In [15]:
#deltaPath = "hdfs://localhost:9000/user/train/sc2_delta"
#deltaPath = "file:///home/train/dataops6/delta"

In [16]:
customers.write \
.mode("overwrite") \
.format("delta") \
.save(deltaPath)

In [6]:
BUCKET = 'tmdb-bronze'
HTTP = 'http://'
ENDPOINT = 'localhost:9000'
AWS_ACCESS_KEY_ID = 'root'
AWS_SECRET_ACCESS_KEY = 'root12345'
REGIONNAME = 'whatever'

session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=None,
    region_name=REGIONNAME,
    botocore_session=None,
    profile_name=None)

s3client = session.client(
    's3', endpoint_url=f"{HTTP}{ENDPOINT}")

s3resource = session.resource(
    's3', endpoint_url=f"{HTTP}{ENDPOINT}")


my_bucket = s3resource.Bucket('tmdb-bronze')
file_names = []
for my_bucket_object in my_bucket.objects.all():
    key=my_bucket_object.key
    if key.find(f"credits/credits_part_{current_ts}-") == 0:
        file_names.append(key[-14:-8])
file_names

['180645', '180737']

In [2]:
spark.stop()

In [1]:
my_bucket = s3resource.Bucket('tmdb-silver')
for my_bucket_object in my_bucket.objects.all():
    key=my_bucket_object.key


SyntaxError: unexpected EOF while parsing (1256331340.py, line 5)

In [60]:
credits.write.mode("overwrite").format("delta").save(f"s3a://tmdb-silver/{current_ts}/crew")

In [26]:
# read from delta minio
cro = spark.read.format("delta").load(f"s3a://tmdb-silver/cast/")
cro.count()

106300

In [19]:
cro = spark.read.format("delta").load(f"s3a://tmdb-silver/cast/")
cro.count()

2022-11-07 22:28:31,785 ERROR delta.DeltaLog: Change in the table id detected while updating snapshot. 
Previous snapshot = Snapshot(path=s3a://tmdb-silver/cast/_delta_log, version=1, metadata=Metadata(b6873935-de3a-498a-b7a9-46755cc2c213,null,null,Format(parquet,Map()),{"type":"struct","fields":[{"name":"movie_id","type":"long","nullable":true,"metadata":{}},{"name":"title","type":"string","nullable":true,"metadata":{}},{"name":"cast_id","type":"integer","nullable":true,"metadata":{}},{"name":"character","type":"string","nullable":true,"metadata":{}},{"name":"credit_id","type":"string","nullable":true,"metadata":{}},{"name":"gender","type":"integer","nullable":true,"metadata":{}},{"name":"id","type":"integer","nullable":true,"metadata":{}},{"name":"name","type":"string","nullable":true,"metadata":{}}]},List(),Map(),Some(1667761028839)), logSegment=LogSegment(s3a://tmdb-silver/cast/_delta_log,1,WrappedArray(S3AFileStatus{path=s3a://tmdb-silver/cast/_delta_log/00000000000000000000.json;

106300

In [68]:
# files in delta logs
silver_bucket = s3resource.Bucket('tmdb-silver')
delta_log= silver_bucket.objects.filter(Prefix=f"{current_ts}/crew/part")
delta_log_list = list(delta_log)
len(list(delta_log))

In [69]:
delta_log_list

[s3.ObjectSummary(bucket_name='tmdb-silver', key='20221105/crew/part-00000-22afa9cc-71bb-4e07-8112-c1be72bb1281-c000.snappy.parquet'),
 s3.ObjectSummary(bucket_name='tmdb-silver', key='20221105/crew/part-00000-53b3b281-2c0d-46fc-b61a-b5059ed7cd1c-c000.snappy.parquet'),
 s3.ObjectSummary(bucket_name='tmdb-silver', key='20221105/crew/part-00000-c0c55544-7345-483a-8ed2-b05c72e7cffc-c000.snappy.parquet')]

In [ ]:
# if there is a file in bucket
silver_bucket = s3resource.Bucket('tmdb-silver')
delta_log= silver_bucket.objects.filter(Prefix=f"{current_ts}/crew/part")
len(list(delta_log))

In [87]:
cro = spark.read.format("parquet").load("tmdb-silver/20221105/cast/part-00000-96f667ee-8636-4417-813c-324272c22dbe-c000.snappy.parquet")
cro.show(5)

2022-11-05 23:31:04,679 WARN net.NetUtils: Unable to wrap exception of type class org.apache.hadoop.ipc.RpcException: it has no (String) constructor
java.lang.NoSuchMethodException: org.apache.hadoop.ipc.RpcException.<init>(java.lang.String)
	at java.base/java.lang.Class.getConstructor0(Class.java:3349)
	at java.base/java.lang.Class.getConstructor(Class.java:2151)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:830)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:806)
	at org.apache.hadoop.ipc.Client.getRpcResponse(Client.java:1515)
	at org.apache.hadoop.ipc.Client.call(Client.java:1457)
	at org.apache.hadoop.ipc.Client.call(Client.java:1367)
	at org.apache.hadoop.ipc.ProtobufRpcEngine$Invoker.invoke(ProtobufRpcEngine.java:228)
	at org.apache.hadoop.ipc.ProtobufRpcEngine$Invoker.invoke(ProtobufRpcEngine.java:116)
	at com.sun.proxy.$Proxy39.getFileInfo(Unknown Source)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolTranslatorPB.getFileInfo(Cli

Py4JJavaError: An error occurred while calling o649.load.
: java.io.IOException: Failed on local exception: org.apache.hadoop.ipc.RpcException: RPC response exceeds maximum data length; Host Details : local host is: "trainvm.vbo.local/127.0.0.1"; destination host is: "localhost":9000; 
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:816)
	at org.apache.hadoop.ipc.Client.getRpcResponse(Client.java:1515)
	at org.apache.hadoop.ipc.Client.call(Client.java:1457)
	at org.apache.hadoop.ipc.Client.call(Client.java:1367)
	at org.apache.hadoop.ipc.ProtobufRpcEngine$Invoker.invoke(ProtobufRpcEngine.java:228)
	at org.apache.hadoop.ipc.ProtobufRpcEngine$Invoker.invoke(ProtobufRpcEngine.java:116)
	at com.sun.proxy.$Proxy39.getFileInfo(Unknown Source)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolTranslatorPB.getFileInfo(ClientNamenodeProtocolTranslatorPB.java:903)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.interna

In [27]:
def a(y):
    print("a function is called")
    x = y+5
    return x,a.__name__

def b(y):
    print("b function is called")
    x = y+5
    return x,b.__name__
liste = [a,b]
str(a)

'<function a at 0x7f9bc571b1f0>'

In [28]:
a.__name__

'a'

In [5]:
yesterday_table = spark.read.format("delta").load(f"s3a://tmdb-silver/cast/")

In [6]:
yesterday_table.limit(5).toPandas()

,movie_id,title,cast_id,character,credit_id,gender,id,name
0,19995,Avatar,242,Jake Sully,5602a8a7c3a3685532001c9a,2,65731,Sam Worthington
1,19995,Avatar,3,Neytiri,52fe48009251416c750ac9cb,1,8691,Zoe Saldana
2,19995,Avatar,25,Dr. Grace Augustine,52fe48009251416c750aca39,1,10205,Sigourney Weaver
3,19995,Avatar,4,Col. Quaritch,52fe48009251416c750ac9cf,2,32747,Stephen Lang
4,19995,Avatar,5,Trudy Chacon,52fe48009251416c750ac9d3,1,17647,Michelle Rodriguez


In [13]:
yesterday_table.count()

212600

In [18]:
yesterday_table.select("character").distinct().count()

71578

# Silver bucket controls

In [82]:
cast= spark.read.format("delta").load(f"s3a://tmdb-silver/cast/")
cast.count()

106300

In [59]:
# Cast

In [83]:
cast.filter(col("credit_id").isNull()).show()

+--------+-----+-------+---------+---------+------+---+----+
|movie_id|title|cast_id|character|credit_id|gender| id|name|
+--------+-----+-------+---------+---------+------+---+----+
+--------+-----+-------+---------+---------+------+---+----+



In [46]:
cast = cast.na.fill(value="0000000000", subset=["credit_id"])

In [84]:
cast.filter(col("credit_id")== "0000000000").show()

+--------+--------------------+-------+---------+----------+------+----+----+
|movie_id|               title|cast_id|character| credit_id|gender|  id|name|
+--------+--------------------+-------+---------+----------+------+----+----+
|  447027|     Running Forever|   null|     null|0000000000|  null|null|null|
|  346081|           Sardaarji|   null|     null|0000000000|  null|null|null|
|  126509|2016: Obama's Ame...|   null|     null|0000000000|  null|null|null|
|  371085|           Sharkskin|   null|     null|0000000000|  null|null|null|
|  325140|Hum To Mohabbat K...|   null|     null|0000000000|  null|null|null|
|  361505|Me You and Five B...|   null|     null|0000000000|  null|null|null|
|  114065|Down & Out With T...|   null|     null|0000000000|  null|null|null|
|  137955|           Crowsnest|   null|     null|0000000000|  null|null|null|
|  102840|  Sex With Strangers|   null|     null|0000000000|  null|null|null|
|   43630| The Helix... Loaded|   null|     null|0000000000|  nu

In [85]:
crew= spark.read.format("delta").load(f"s3a://tmdb-silver/crew/")
crew.count()

129609

In [79]:
crew.filter(col("credit_id").isNull()).show()

+--------+-----+---------+----------+------+---+---+----+
|movie_id|title|credit_id|department|gender| id|job|name|
+--------+-----+---------+----------+------+---+---+----+
+--------+-----+---------+----------+------+---+---+----+



In [80]:
crew.filter(col("credit_id")=="0000000000").show()

+--------+--------------------+----------+----------+------+----+----+----+
|movie_id|               title| credit_id|department|gender|  id| job|name|
+--------+--------------------+----------+----------+------+----+----+----+
|  371085|           Sharkskin|0000000000|      null|  null|null|null|null|
|   48382|The Book of Mormo...|0000000000|      null|  null|null|null|null|
|  325140|Hum To Mohabbat K...|0000000000|      null|  null|null|null|null|
|   20653|      Roadside Romeo|0000000000|      null|  null|null|null|null|
|  361505|Me You and Five B...|0000000000|      null|  null|null|null|null|
|  114065|Down & Out With T...|0000000000|      null|  null|null|null|null|
|  137955|           Crowsnest|0000000000|      null|  null|null|null|null|
|  102840|  Sex With Strangers|0000000000|      null|  null|null|null|null|
|   43630| The Helix... Loaded|0000000000|      null|  null|null|null|null|
|  357441|   Karachi se Lahore|0000000000|      null|  null|null|null|null|
|  279759| H

In [56]:
# Genres

In [51]:
genres= spark.read.format("delta").load(f"s3a://tmdb-silver/genres/")
genres.count()

12188

In [55]:
genres.filter(col("id")== -9999).show()

+--------+-----+----+
|movie_id|   id|name|
+--------+-----+----+
|  191229|-9999|null|
|  206412|-9999|null|
|  371085|-9999|null|
|  346081|-9999|null|
|  335874|-9999|null|
|  281189|-9999|null|
|  137955|-9999|null|
|  357834|-9999|null|
|  380097|-9999|null|
|  282128|-9999|null|
|  162396|-9999|null|
|  328307|-9999|null|
|   48382|-9999|null|
|  126186|-9999|null|
|  198370|-9999|null|
|  325579|-9999|null|
|  294550|-9999|null|
|  219716|-9999|null|
|  279759|-9999|null|
|  176074|-9999|null|
+--------+-----+----+
only showing top 20 rows



In [54]:
genres.filter(col("id")== -9999).count()

28

In [61]:
movies= spark.read.format("delta").load(f"s3a://tmdb-silver/movies/")
movies.count()

4803

In [62]:
genres.filter(col("id")== -9999).count()

28

In [72]:
# Spoken Languages

In [68]:
spoken_languages= spark.read.format("delta").load(f"s3a://tmdb-silver/spoken_languages/")
spoken_languages.count()

7023

In [69]:
spoken_languages.filter(col("iso_639_1")== "XX").count()

86

In [71]:
spoken_languages.filter(col("iso_639_1").isNull()).show()

+--------+---------+----+
|movie_id|iso_639_1|name|
+--------+---------+----+
+--------+---------+----+



In [73]:
production_countries= spark.read.format("delta").load(f"s3a://tmdb-silver/production_countries/")
production_countries.count()

6610

In [77]:
production_countries.filter(col("iso_3166_1").isNull()).show()
production_countries.printSchema()

+--------+----------+----+
|movie_id|iso_3166_1|name|
+--------+----------+----+
+--------+----------+----+

root
 |-- movie_id: string (nullable = true)
 |-- iso_3166_1: string (nullable = true)
 |-- name: string (nullable = true)



In [76]:
production_countries.filter(col("iso_3166_1")=="XX").show()

+--------+----------+----+
|movie_id|iso_3166_1|name|
+--------+----------+----+
|   24113|        XX|null|
|   36597|        XX|null|
|  150211|        XX|null|
|  281189|        XX|null|
|   16433|        XX|null|
|  170480|        XX|null|
|  371085|        XX|null|
|   30979|        XX|null|
|  325123|        XX|null|
|  219716|        XX|null|
|  323270|        XX|null|
|  206412|        XX|null|
|  320435|        XX|null|
|  323271|        XX|null|
|   29731|        XX|null|
|   43653|        XX|null|
|   77332|        XX|null|
|   84659|        XX|null|
|  279759|        XX|null|
|   65448|        XX|null|
+--------+----------+----+
only showing top 20 rows

